In [ ]:
import deepchem as dc
from deepchem.models import AttentiveFPModel
import pandas as pd
import torch
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import deepchem as dc
from deepchem.models import GCNModel
from sklearn.metrics import r2_score

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\manan\AppData\Local\Programs\Python\Python311\Lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
data = pd.read_csv("XY.csv")
df = pd.DataFrame(data)
df.head()

,Smiles,AlogP,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI
0,C[C@@H](Oc1cccc2ncnc(Nc3ccc4c(cnn4Cc4nccs4)c3)...,4.08,16.02,0.30,3.51,7.74
1,C[C@@H](Oc1cccc2ncnc(Nc3ccc4c(cnn4Cc4cscn4)c3)...,3.48,15.38,0.29,3.37,5.67
2,C[C@H](Oc1cccc2ncnc(Nc3ccc4c(cnn4Cc4cscn4)c3)c...,4.08,14.54,0.28,2.81,7.02
3,CN/N=C/c1c(N)ncnc1Nc1ccc2c(cnn2Cc2cccc(F)c2)c1,2.89,16.50,0.30,3.55,6.08
4,Cc1ccc(Oc2ccc(Nc3ncnc4cccc(O[C@H](C)C(=O)NCCO)...,4.05,14.15,0.26,2.65,5.65


In [3]:
featurizer = dc.feat.CircularFingerprint() 
features = featurizer.featurize(df["Smiles"]) #fingerprints is representation of smiles features in encoded form

In [5]:
X_train,X_test,y_train,y_test = train_test_split(features,df["Ligand Efficiency BEI"],random_state=43)
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

KNN

In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [7]:
knn = KNeighborsRegressor(n_neighbors=3)  # Specify the number of neighbors
    
# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)


R2 Score: 0.5233843816528629
MAE using sklearn: 1.4571721311475412
MSE using sklearn: 4.380876730418945


DT

In [8]:
from sklearn.tree import DecisionTreeRegressor

In [9]:
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)

DecisionTreeRegressor()

In [10]:
y_pred = dt.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
r2_score(y_test,y_pred)

MAE using sklearn: 1.4962462301742931
MSE using sklearn: 5.40101237263957


0.41239915886882905

Random Forest

In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
rf = RandomForestRegressor()

In [13]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [14]:
y_pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
r2_score(y_test,y_pred)

MAE using sklearn: 1.3072102038432354
MSE using sklearn: 3.2731601938274713


0.6438979305448003

GradientBoosting

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [17]:
gb_regressor.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [18]:
y_pred = gb_regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
print("r2-score " ,r2_score(y_test,y_pred))

MAE using sklearn: 1.5941852058534962
MSE using sklearn: 4.328399615297526
r2-score  0.5290936070458078


Adaboost

In [19]:
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [20]:

adaboost_reg = AdaBoostRegressor(n_estimators=100, random_state=42)


In [21]:
adaboost_reg.fit(X_train,y_train)

AdaBoostRegressor(n_estimators=100, random_state=42)

In [22]:
y_pred = adaboost_reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
print("r2-score " ,r2_score(y_test,y_pred))

MAE using sklearn: 2.1829855561173983
MSE using sklearn: 7.413047157329429
r2-score  0.19350069126707115


XGBOOST

In [31]:
import xgboost as xgb

In [32]:
# Define the XGBoost regressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit the XGBoost regressor to the training data
xgb_reg.fit(X_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [33]:
y_pred= xgb_reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
print("r2-score " ,r2_score(y_test,y_pred))


MAE using sklearn: 1.2518011689576947
MSE using sklearn: 3.0958784591317268
r2-score  0.6631852213779266


SVR reg

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [35]:
svr_regressor = SVR(kernel='linear')

# Fit the regressor to the training data
svr_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = svr_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
print("r2-score " ,r2_score(y_test,y_pred))

MAE using sklearn: 1.3028431740387592
MSE using sklearn: 3.30167409465346
r2-score  0.6407957728467069


Huber reg

In [36]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import HuberRegressor

In [37]:

huber_regressor = HuberRegressor()
# Fit the regressor to the training data
huber_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = huber_regressor.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("MAE using sklearn:", mae)
print("MSE using sklearn:", mse)
print("r2-score " ,r2_score(y_test,y_pred))


MAE using sklearn: 1.3362656637426
MSE using sklearn: 3.513554948697535
r2-score  0.6177442855576594


c:\Users\manan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
